In [32]:
import pandas as pd
import numpy as np
import scorecardpy as sc
from sklearn import tree
from sklearn import metrics
from matplotlib import pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

In [10]:
loan_data = pd.read_excel('data_v4.xlsx')

In [12]:
loan_data.shape

(50000, 7)

In [18]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sample_id                  50000 non-null  int64  
 1   sample_month               50000 non-null  int64  
 2   adr_stability_grade        50000 non-null  int64  
 3   ovd_order_cnt_6m_grade     50000 non-null  int64  
 4   positive_biz_cnt_1y_grade  50000 non-null  int64  
 5   risk_score                 50000 non-null  int64  
 6   is_dlq_30d                 47430 non-null  float64
dtypes: float64(1), int64(6)
memory usage: 2.7 MB


In [14]:
loan_data.head()

,sample_id,sample_month,adr_stability_grade,ovd_order_cnt_6m_grade,positive_biz_cnt_1y_grade,risk_score,is_dlq_30d
0,1,201802,10,1,10,1,0.0
1,2,201804,10,1,7,2,0.0
2,3,201803,10,1,7,3,0.0
3,4,201802,10,1,9,3,0.0
4,5,201804,10,1,10,1,0.0


Suppose that our existing rules are "adr_stability_grade" and "positive_biz_cnt_1y_grade"

The new variables coming from the thrid party are "ovd_order_cnt_6m_grade" and "risk score"

## Select The Approval Sample 

In [60]:
data_prep = loan_data.copy()

## Select the sample
data_prep_appr = data_prep[data_prep['is_dlq_30d'].isnull() == False]
data_prep_appr = data_prep_appr[['sample_id', 'sample_month', 'adr_stability_grade', 'positive_biz_cnt_1y_grade', 'is_dlq_30d']]

In [62]:
## Change the type of target variable to int
data_prep_appr['is_dlq_30d'] = data_prep_appr['is_dlq_30d'].astype(int)

## Calculate the bad accounts concentration
badrate_concen_old = data_prep_appr['is_dlq_30d'].mean()
print('The bad rate concentration based on the old underwriting rules is {}'.format(badrate_concen_old))

The bad rate concentration based on the old underwriting rules is 0.09502424625764284


## Check The Bins Information for Each Variable in Old rules

In [64]:
data_prep_appr

,sample_id,sample_month,adr_stability_grade,positive_biz_cnt_1y_grade,is_dlq_30d
0,1,201802,10,10,0
1,2,201804,10,7,0
2,3,201803,10,7,0
3,4,201802,10,9,0
4,5,201804,10,10,0
...,...,...,...,...,...
49988,49989,201803,4,8,0
49989,49990,201804,4,6,0
49990,49991,201804,2,3,0
49996,49997,201804,2,2,0


In [72]:
var0 = data_prep_appr.columns.difference(['sample_id', 'sample_month']).tolist()

[INFO] creating woe binning ...


In [156]:
def get_bins_info(df, var, target_var):
    bins_info = sc.woebin(df[[var, target_var]], y = target_var, method = 'tree', break_list = {}, stop_limit = 0)
    bins_info[var]['lift'] = bins_info[var]['badprob']/badrate_concen_old
    display(bins_info[var].style.bar(subset=['badprob'], color='lightgreen', width=80, height=80, align='left', vmin=0).highlight_max(subset=['lift'], color='red', axis=0))
    print('The max lift value is {}'.format(bins_info[var]['lift'].max()))

In [158]:
get_bins_info(data_prep_appr, 'adr_stability_grade', 'is_dlq_30d')

[INFO] creating woe binning ...


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values,lift
0,adr_stability_grade,"[-inf,3.0)",5945,0.125343,4885,1060,0.178301,0.725876,0.088108,0.290425,3.0,False,1.876375
1,adr_stability_grade,"[3.0,4.0)",3415,0.072001,2879,536,0.156955,0.572712,0.029697,0.290425,4.0,False,1.651732
2,adr_stability_grade,"[4.0,5.0)",7948,0.167573,6978,970,0.122043,0.280555,0.014771,0.290425,5.0,False,1.284338
3,adr_stability_grade,"[5.0,6.0)",4187,0.088277,3778,409,0.097683,0.030541,0.000083,0.290425,6.0,False,1.027983
4,adr_stability_grade,"[6.0,7.0)",3690,0.077799,3371,319,0.086450,-0.103998,0.000807,0.290425,7.0,False,0.909766
5,adr_stability_grade,"[7.0,8.0)",6848,0.144381,6352,496,0.072430,-0.296173,0.011235,0.290425,8.0,False,0.762226
6,adr_stability_grade,"[8.0,9.0)",5551,0.117036,5242,309,0.055666,-0.577341,0.030926,0.290425,9.0,False,0.585805
7,adr_stability_grade,"[9.0,inf)",9846,0.207590,9438,408,0.041438,-0.887456,0.114798,0.290425,inf,False,0.436080


The max lift value is 1.876374718851509


In [160]:
get_bins_info(data_prep_appr, 'positive_biz_cnt_1y_grade', 'is_dlq_30d')

[INFO] creating woe binning ...


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values,lift
0,positive_biz_cnt_1y_grade,"[-inf,3.0)",5636,0.118828,4572,1064,0.188786,0.795861,0.103112,0.329925,3.0,False,1.986718
1,positive_biz_cnt_1y_grade,"[3.0,4.0)",2690,0.056715,2233,457,0.169888,0.667358,0.032950,0.329925,4.0,False,1.787843
2,positive_biz_cnt_1y_grade,"[4.0,5.0)",3157,0.066561,2714,443,0.140323,0.441167,0.015468,0.329925,5.0,False,1.476708
3,positive_biz_cnt_1y_grade,"[5.0,7.0)",6978,0.147122,6167,811,0.116222,0.225076,0.008163,0.329925,7.0,False,1.223082
4,positive_biz_cnt_1y_grade,"[7.0,8.0)",3639,0.076724,3270,369,0.101401,0.072027,0.000410,0.329925,8.0,False,1.067112
5,positive_biz_cnt_1y_grade,"[8.0,9.0)",3844,0.081046,3552,292,0.075963,-0.244736,0.004397,0.329925,9.0,False,0.799402
6,positive_biz_cnt_1y_grade,"[9.0,10.0)",4734,0.099810,4486,248,0.052387,-0.641512,0.031747,0.329925,10.0,False,0.551301
7,positive_biz_cnt_1y_grade,"[10.0,inf)",16752,0.353194,15929,823,0.049128,-0.709164,0.133679,0.329925,inf,False,0.517010


The max lift value is 1.9867179246288753


The lift values of these two existing variables are less than 2, there is no reason we can tighen their cutoff.

## Check The New Variables Coming from The Third Party (IV and Woe and The Order of Bad Rates)

In [164]:
var1 = ['risk_score','ovd_order_cnt_6m_grade','is_dlq_30d']

data_prep_appr2 = data_prep[data_prep['is_dlq_30d'].isnull() == False]
data_prep_appr2['is_dlq_30d'] = data_prep_appr2['is_dlq_30d'].astype(int)

get_bins_info(data_prep_appr2, 'risk_score', 'is_dlq_30d')

[INFO] creating woe binning ...


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values,lift
0,risk_score,"[-inf,2.0)",10724,0.226102,10690,34,0.003170,-3.496927,0.844532,2.414019,2.0,False,0.033365
1,risk_score,"[2.0,3.0)",7337,0.154691,7244,93,0.012675,-2.101553,0.311309,2.414019,3.0,False,0.133392
2,risk_score,"[3.0,4.0)",8062,0.169977,7768,294,0.036467,-1.020412,0.118106,2.414019,4.0,False,0.383769
3,risk_score,"[4.0,5.0)",7658,0.161459,7169,489,0.063855,-0.431383,0.025245,2.414019,5.0,False,0.671984
4,risk_score,"[5.0,6.0)",7578,0.159772,6560,1018,0.134336,0.390625,0.028531,2.414019,6.0,False,1.413705
5,risk_score,"[6.0,7.0)",2822,0.059498,2082,740,0.262225,1.219342,0.141058,2.414019,7.0,False,2.759563
6,risk_score,"[7.0,inf)",3249,0.068501,1410,1839,0.566020,2.519408,0.945238,2.414019,inf,False,5.956588


The max lift value is 5.956588304926716


In [166]:
get_bins_info(data_prep_appr2, 'ovd_order_cnt_6m_grade', 'is_dlq_30d')

[INFO] creating woe binning ...


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values,lift
0,ovd_order_cnt_6m_grade,"[-inf,2.0)",37057,0.781299,35465,1592,0.042961,-0.849779,0.401961,1.077085,2.0,False,0.452104
1,ovd_order_cnt_6m_grade,"[2.0,3.0)",4566,0.096268,3629,937,0.205212,0.899747,0.110986,1.077085,3.0,False,2.159580
2,ovd_order_cnt_6m_grade,"[3.0,4.0)",3007,0.063399,2089,918,0.305288,1.431533,0.221908,1.077085,4.0,False,3.212734
3,ovd_order_cnt_6m_grade,"[4.0,inf)",2800,0.059034,1740,1060,0.378571,1.758160,0.342229,1.077085,inf,False,3.983946


The max lift value is 3.98394560841865


We can found that the relationship between risk score and the target variable is strong and it has a high value of lift

## Try to Add Risk Score as A Single Rating

In [402]:
# Create a function to calculate different types of hit rate (Genral hit, Single hit rate, pure/base hit rate and pure hit rate percentage)

def cal_hit(df, rule_set):
    '''df : dataframe'''
    '''rule_set : a list including 3 elements tuples (rule name, value and relation)'''
    
    ## We first create a temp dataframe with the hit situation by the rules
    for r in rule_set:
        if r[2] == '>':
            df[r[0] + '_hit'] = np.where(df[r[0]] > r[1], 1, 0)
        elif r[2] == '>=':
            df[r[0] + '_hit'] = np.where(df[r[0]] >= r[1], 1, 0)
        elif r[2] == '<':
            df[r[0] + '_hit'] = np.where(df[r[0]] < r[1], 1, 0)
        elif r[2] == '<=':
            df[r[0] + '_hit'] = np.where(df[r[0]] <= r[1], 1, 0)

    ## Based on the temp dataframe, we then can calculate the general hit rate 
    rule_cols = [r for r in df.columns if '_hit' in r]
    df['hit_any'] = df[rule_cols].any(axis = 1)
    print('''####### General Hit Rate ########''')
    print('The General Hit Rate is {}%'.format(df['hit_any'].mean()*100))

    df['hit_sum'] = df[rule_cols].sum(axis = 1)
    print('''####### Hit Distribution #######''')
    print(df['hit_sum'].value_counts()/df.shape[0])

    ## Then we calculate different hit rates
    single_hit_rate = []
    pure_hit_rate = []
    pure_rate_pct = []
    for i in rule_cols:
        ## single hit rate
        single_hit_rate.append(df[i].mean())
        ## pure hit rate
        pure_hit_rate.append(df[(df['hit_sum'] == 1) & (df[i] == 1)].shape[0]/df.shape[0])
        # pure rate percentage
        pure_rate_pct.append((df[(df['hit_sum'] == 1) & (df[i] == 1)].shape[0]/df.shape[0])/df['hit_any'].mean())

    ## Combine them to a dataframe
    hit_rate_df = pd.DataFrame({'rule' : rule_cols, 'single_hit' : single_hit_rate, 'pure_hit' : pure_hit_rate, 'pure_pct' : pure_rate_pct})

    return hit_rate_df, df

In [404]:
# We first select the sample, since we aim to calculating the hit rate, we may not need the behaviral data of the borrowers, so we can select those are approved and rejected

data_prep_all = loan_data.copy()

In [406]:
# Setting the rule set (The Old One)
rule_set_0 = []

rule0 = ('adr_stability_grade', 2, '<')
rule1 = ('positive_biz_cnt_1y_grade', 2, '<')

rule_set_0.append(rule0)
rule_set_0.append(rule1)

In [408]:
hit_rate_df, data_temp_old = cal_hit(data_prep_all, rule_set_0)

####### General Hit Rate ########
The General Hit Rate is 5.140000000000001%
####### Hit Distribution #######
hit_sum
0    0.94860
1    0.05076
2    0.00064
Name: count, dtype: float64


In [410]:
data_final_old

,sample_id,sample_month,adr_stability_grade,ovd_order_cnt_6m_grade,positive_biz_cnt_1y_grade,risk_score,is_dlq_30d,adr_stability_grade_hit,positive_biz_cnt_1y_grade_hit,hit_any,hit_sum
0,1,201802,10,1,10,1,0.0,0,0,False,0
1,2,201804,10,1,7,2,0.0,0,0,False,0
2,3,201803,10,1,7,3,0.0,0,0,False,0
3,4,201802,10,1,9,3,0.0,0,0,False,0
4,5,201804,10,1,10,1,0.0,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,201803,1,1,2,4,NaN,1,0,True,1
49996,49997,201804,2,1,2,8,0.0,0,0,False,0
49997,49998,201804,1,2,1,8,NaN,1,1,True,2
49998,49999,201804,1,1,2,5,NaN,1,0,True,1


In [412]:
# Setting the rule set (The New One)

rule_set = []

rule0 = ('adr_stability_grade', 2, '<')
rule1 = ('positive_biz_cnt_1y_grade', 2, '<')
rule2 = ('risk_score', 6, '>=')

rule_set.append(rule0)
rule_set.append(rule1)
rule_set.append(rule2)


In [427]:
data_prep_all = loan_data.copy()
hit_rate_df_new, data_temp_new = cal_hit(data_prep_all, rule_set)

####### General Hit Rate ########
The General Hit Rate is 17.282%
####### Hit Distribution #######
hit_sum
0    0.82718
1    0.15684
2    0.01566
3    0.00032
Name: count, dtype: float64


In [429]:
hit_rate_df_new

,rule,single_hit,pure_hit,pure_pct
0,adr_stability_grade_hit,0.03708,0.02600,0.150446
1,positive_biz_cnt_1y_grade_hit,0.01496,0.00942,0.054508
2,risk_score_hit,0.13708,0.12142,0.702581


## Calculate The Default Rate 

In [432]:
data_temp_new.isnull().sum()

sample_id                           0
sample_month                        0
adr_stability_grade                 0
ovd_order_cnt_6m_grade              0
positive_biz_cnt_1y_grade           0
risk_score                          0
is_dlq_30d                       2570
adr_stability_grade_hit             0
positive_biz_cnt_1y_grade_hit       0
risk_score_hit                      0
hit_any                             0
hit_sum                             0
dtype: int64

In [434]:
# Re-select the sample with behaviral data

data_temp_new = data_final_new[data_final_new['is_dlq_30d'].isnull() == False]

In [436]:
data_temp_new.isnull().sum()

sample_id                        0
sample_month                     0
adr_stability_grade              0
ovd_order_cnt_6m_grade           0
positive_biz_cnt_1y_grade        0
risk_score                       0
is_dlq_30d                       0
adr_stability_grade_hit          0
positive_biz_cnt_1y_grade_hit    0
risk_score_hit                   0
hit_any                          0
hit_sum                          0
dtype: int64

In [438]:
data_final = data_temp_new[data_temp_new['hit_sum'] == 0]

In [440]:
badrate_concen_new = data_final['is_dlq_30d'].mean()

In [442]:
badrate_concen_new

0.04661621412509974